# 随机梯度下降算法

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

In [2]:
data = pd.read_csv('data/Exampe1_D40.csv')
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X31,X32,X33,X34,X35,X36,X37,X38,X39,y
0,1.250107,0.432554,-0.578400,-1.513010,-0.669289,-0.608792,0.804890,0.477470,1.346126,-0.095303,...,-1.177931,1.459452,1.064232,0.473094,1.286367,1.060472,-1.094578,1.355803,0.123127,-1.477687
1,1.058146,1.377502,-0.623408,-1.351336,-0.956631,-0.258621,1.111579,1.248884,-1.707337,0.039080,...,-0.906663,-1.256201,-0.042359,1.684977,-0.900141,0.604080,0.910155,-0.909194,0.774623,0.320275
2,-0.458058,0.463756,0.477930,0.129045,-1.432883,1.143771,-0.613325,-1.081578,-1.590011,0.318591,...,1.370581,1.358135,0.456469,-0.539681,-0.527895,0.790820,1.382501,1.336876,0.952814,0.345420
3,0.487717,-1.435076,-1.169231,1.390551,0.352955,-1.694393,-1.374105,0.567149,-1.713895,-1.181347,...,0.004282,0.238679,-0.031716,-1.034371,0.768722,-0.753833,-1.660227,0.501272,-1.126344,0.486485
4,1.516874,1.577849,1.459923,-0.446858,-1.691805,1.463337,-0.240741,1.615019,1.611169,1.232733,...,1.068974,1.242749,1.429268,0.053149,0.001207,1.041816,0.520902,0.696654,1.007719,0.816467


In [6]:
features = data.columns[:-1]
X = data[features].values
y = data['y'].values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

print(X.shape, y.shape)

torch.Size([10000, 40]) torch.Size([10000, 1])


In [7]:
# Split the data into training and testing sets
len = X.shape[0]
train_len = int(len * 0.8)

X_train = X[:train_len]
y_train = y[:train_len]

X_test = X[train_len:]
y_test = y[train_len:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([8000, 40]),
 torch.Size([8000, 1]),
 torch.Size([2000, 40]),
 torch.Size([2000, 1]))

In [10]:
# Create a DataLoader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 40]) torch.Size([64, 1])


In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(40, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 1, bias=True)
        self.tanh = nn.Tanh()

    def forward(self, x):
        fc1 = self.tanh(self.fc1(x))
        fc2 = self.fc2(fc1)
        div_value = 1000
        y = fc2 / div_value
        return y

In [ ]:
def relative_error(y_pred, y_true):
    return torch.sum(torch.abs(y_pred - y_true) / torch.abs(y_true))

In [17]:
model = MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.MSELoss()
# 定义相对误差

train_losses = []
val_losses = []
train_RE = []
val_RE = []

epochs = 1000

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_relative_error = 0.0
    batch_count = 0
    for X, y in train_loader:
        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_relative_error += relative_error(y_pred, y).item()
        batch_count += 1
    train_losses.append(train_loss / batch_count)
    train_RE.append(train_relative_error / batch_count)
    
    model.eval()
    with torch.no_grad():
        val_Pred = model(X_test)
        val_loss = criterion(val_Pred, y_test)
        val_relative_error = relative_error(val_Pred, y_test)
    val_losses.append(val_loss)
    val_RE.append(val_relative_error)
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train RE: {train_RE[-1]:.4f}, Val RE: {val_RE[-1]:.4f}')

Epoch 1/1000, Train Loss: 1.0048, Val Loss: 0.9808, Train RE: 1.0001, Val RE: 1.0008
Epoch 2/1000, Train Loss: 1.0047, Val Loss: 0.9808, Train RE: 1.0000, Val RE: 1.0008
Epoch 3/1000, Train Loss: 1.0047, Val Loss: 0.9808, Train RE: 1.0000, Val RE: 1.0007
Epoch 4/1000, Train Loss: 1.0047, Val Loss: 0.9808, Train RE: 1.0000, Val RE: 1.0007
Epoch 5/1000, Train Loss: 1.0047, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 6/1000, Train Loss: 1.0047, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 7/1000, Train Loss: 1.0047, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 8/1000, Train Loss: 1.0046, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 9/1000, Train Loss: 1.0046, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 10/1000, Train Loss: 1.0046, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 11/1000, Train Loss: 1.0046, Val Loss: 0.9807, Train RE: 1.0000, Val RE: 1.0007
Epoch 12/1000, Train Loss: 1.0046, Val Loss: 0.9806, Train RE: 

Epoch 1/100, Train Loss: 1.0046, Val Loss: 0.9806, Train RE: 0.9999, Val RE: 1.0050
